**В нашей выборке 100 отзывов на мобильные телефоны, на старые модели, а будем мы оценивать мы давать им оценку по отзывам на новые модели (относительность времени доказана Эйнштейном). Будем парсить отзывы с мэйла, так как на яндексе не все подводят резюме, искать где его написали, а где нет дело неблагодарное, а использование в оценке блоки с недостатками и блоки с достоинствами должно только запутать наш классификатор. Плюс на мэйле отзывы идут подряд на все модели, ограничены только маркой в отличии от яндекс маркета**

**Будем парсить отзывы на все телефоны вот ссылка https://torg.mail.ru/review/goods/mobilephones/. Их более 17000. Отзывы с оценкой 1,2,3 будут отрицательными (neg,-1), а с оценками 4,5 - положительными (pos,1). К сожалению другого нам не дано**

**Проблема 1. Ссылки "Читать полностью". Надо будет проверять на наличие такой ссылки, и если она есть, то брать отзыв оттуда, а не из основного класса**

In [1]:
# ячейка для испортирования библиотек
import requests
import bs4
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import pandas as pd
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.pipeline import Pipeline

In [105]:
# напишем функцию для парсинга одной страницы
# она будет возвращать тексты отзывов со страницы и метки классов, принимая на вход тексты страниц
def parsing(html):
    # создаем два массива для текстов и меток
    texts = []
    labels = []
    # сначала находим все отзывы на телефоны на 
    soup = bs4.BeautifulSoup(html)
    reviews = soup.find_all('div',{'class':'review-item__body'})
    # теперь для каждой рецензии
    for review in reviews:
        # находим оценку
        grade = review.find('span', {'class': 'review-item__rating-counter'})
        grade = grade.text
        
        # поправка к функции (смотри далее)
        if len(grade) == 1:
            grade = float(grade)
            
            # присваиваем ей класс
            if grade > 3:
                labels.append(1)
            else:
                labels.append(-1)
                # продолжаем в этом if-e. Дальше находим наш отзыв без недостатков и достоинтсв отдельно
                # он всегда пишется первым на этом сайте, поэтому используем просто find
            part = review.find('p', {'class': 'review-item__paragraph'})
                # теперь надо проверить есть ли ссылка "Читать полностью"
            link = part.find_all('a')
                # если ссылок нет загружаем текст отзыва
            if len(link) == 0:
                texts.append(part.text.strip())
                # если есть ссылка, достаем текст из нее
            else:
                txt = link[0].get('full-text')
                texts.append(txt)
    return texts, labels

**Проверим функции на первой страничке отзывов**

In [106]:
url = 'https://torg.mail.ru/review/goods/mobilephones/'
r = requests.get(url)

In [107]:
txts, labs = parsing(r.text)

In [108]:
len(labs), len(txts)

(20, 20)

**Все правильно **

In [109]:
print labs

[1, 1, 1, 1, -1, 1, -1, 1, -1, -1, -1, 1, 1, 1, -1, -1, 1, 1, 1, -1]


**Возможная проблема номер 2 - несбалансированность выборки, что видно по нашим меткам. Будем решать по ходу создания модели**

**Попробуем распарсить 100 страниц**

**В ходе выполнения кода столкнулись с проблемой номер 3! Оценка 4,5 не превращается во float. Немного поправим нашу функцию**

In [110]:
num = range(2,100)
for d in num:
    url = 'https://torg.mail.ru/review/goods/mobilephones/?page=%d' % (d)
    r = requests.get(url)
    t,l = parsing(r.text)
    txts.extend(t)
    labs.extend(l)

In [111]:
len(labs)

1979

**Получили 1979 отзывов**

In [112]:
np.unique(labs, return_counts=True)

(array([-1,  1]), array([ 469, 1510], dtype=int64))

**Здесь уже видна несбалансированность классов, примерно 25% отрицательных отзывов. Распарсим дальше еще 700 страниц отзывов**

In [145]:
num = range(100,800)
for d in num:
    url = 'https://torg.mail.ru/review/goods/mobilephones/?page=%d' % (d)
    r = requests.get(url)
    t,l = parsing(r.text)
    txts.extend(t)
    labs.extend(l)

In [146]:
len(labs)

11577

In [147]:
np.unique(labs, return_counts=True)

(array([-1,  1]), array([2374, 9203], dtype=int64))

**Сохраним нашу обучающую выборку**

In [154]:
import pickle
with open('train_texts.pkl', 'wb') as t_pkl:
    pickle.dump(txts, t_pkl, protocol=2)
with open('train_labs.pkl', 'wb') as l_pkl:
    pickle.dump(labs, l_pkl, protocol=2)

**Теперь будем работать с этой выборкой. Первые соображения - лучше не убирать знаки препинания (возможно смайлики, восклицательные и вопросительные знаки говорят о тональности отзыва). Простое предсказание с обычным tfidf и логистической регрессии дало 0.9 accuracy на кегл. Будем пробовать улучшать с помощью grid search и undersampling **

In [51]:
# отзывы от заказчика
with open('test.csv', 'r') as f:
    data = f.read()
data = [r.text for r in bs4.BeautifulSoup(data, 'lxml').find_all('review')]

In [54]:
# функция для записи предсказаний 
def writeanswer(preds, name):
    preds_1 = []
    for i, g in enumerate(preds):
        if g == 1:
            preds_1.append('pos')
        else:
            preds_1.append('neg')
    dat = pd.DataFrame({'y':preds_1, 'Id':range(100) })
    dat.to_csv(name, index=False)

**Сначал проведем grid search на всей выборке среди 3х алгоритмов SGDClassifier, LogisticRegression и LinearSVC.**

In [193]:
tf = TfidfVectorizer(encoding='unicode')
cv = CountVectorizer(encoding='unicode')
lr = LogisticRegression()
sgd = SGDClassifier()
svc = LinearSVC()
skf = StratifiedKFold(n_splits=3, random_state=17, shuffle=True)
clfs = [lr,sgd,svc]
vectors = [tf, cv]

**Найдем сначала лучший векторизатор**

In [195]:
# словесные n-граммы
for i, vector in enumerate(vectors):
    for j, clf in enumerate(clfs):
        parameters_grid = {
            'vectorizer__ngram_range' : [(1,2),(1,3),(1,4), (1,1)],
            'vectorizer__min_df' : [1,0.1,0.05],
            'vectorizer__max_df' : [1.0, 0.95, 0.9]
        }
        pl = Pipeline([('vectorizer', vector),('classifier', clf)])
        grid_cv = GridSearchCV(pl, parameters_grid, scoring = 'accuracy', cv = skf, n_jobs=-1)
        grid_cv.fit(txts, labs)
        print grid_cv.best_params_
        print i,j, grid_cv.best_score_
        print

{'vectorizer__min_df': 1, 'vectorizer__ngram_range': (1, 1), 'vectorizer__max_df': 1.0}
0 0 0.810659065388

{'vectorizer__min_df': 1, 'vectorizer__ngram_range': (1, 2), 'vectorizer__max_df': 0.9}
0 1 0.831303446489

{'vectorizer__min_df': 1, 'vectorizer__ngram_range': (1, 2), 'vectorizer__max_df': 1.0}
0 2 0.829143992399

{'vectorizer__min_df': 1, 'vectorizer__ngram_range': (1, 3), 'vectorizer__max_df': 1.0}
1 0 0.826466269327

{'vectorizer__min_df': 1, 'vectorizer__ngram_range': (1, 3), 'vectorizer__max_df': 0.9}
1 1 0.814891595405

{'vectorizer__min_df': 1, 'vectorizer__ngram_range': (1, 3), 'vectorizer__max_df': 1.0}
1 2 0.801848492701



In [199]:
# буквенные n-граммы
tf1 = TfidfVectorizer(encoding='unicode', analyzer='char')
cv1 = CountVectorizer(encoding='unicode', analyzer='char')
vectors = [tf1, cv1]
for i, vector in enumerate(vectors):
    for j, clf in enumerate(clfs):
        parameters_grid = {
            'vectorizer__ngram_range' : [(2,5),(3,5),(4,6), (3,6)],
        }
        pl = Pipeline([('vectorizer', vector),('classifier', clf)])
        grid_cv = GridSearchCV(pl, parameters_grid, scoring = 'accuracy', cv = skf, n_jobs=-1)
        grid_cv.fit(txts, labs)
        print grid_cv.best_params_
        print i,j, grid_cv.best_score_
        print

{'vectorizer__ngram_range': (2, 5)}
0 0 0.811090956206

{'vectorizer__ngram_range': (2, 5)}
0 1 0.831044311998

{'vectorizer__ngram_range': (2, 5)}
0 2 0.831648959143

{'vectorizer__ngram_range': (3, 6)}
1 0 0.820678932366

{'vectorizer__ngram_range': (4, 6)}
1 1 0.802453139846

{'vectorizer__ngram_range': (4, 6)}
1 2 0.779822060983



**Итак, мы нашли лучшие векторизаторы для различных классификаторов. Логистическая регрессия - CountVectorizer для слов,
SGD - tfidf для слов, SVC - tfidf для букв**

In [207]:
# настройка логистической регрессии
lr = LogisticRegression(random_state=17, n_jobs=-1)
v_for_lr = CountVectorizer(min_df= 1, ngram_range=(1, 3), max_df= 1.0,encoding='unicode')
train = v_for_lr.fit_transform(txts)
params = {
    'C':[1,0.5,0.1,3,5],
    'class_weight':[None, 'balanced']
}
grid_cv = GridSearchCV(lr, params, scoring = 'accuracy', cv = skf, n_jobs=-1)
grid_cv.fit(train, labs)
print grid_cv.best_params_
print i,j, grid_cv.best_score_

{'C': 1, 'class_weight': None}
1 2 0.826466269327


In [209]:
# обучим и запишем предсказания
lr = LogisticRegression(random_state=17, n_jobs=-1, C = 1, class_weight='balanced')
lr.fit(train, labs)
test = v_for_lr.transform(data)
preds = lr.predict(test)
writeanswer(preds, 'otvety_lr.csv')

In [203]:
# натсройка sgd
sgd = SGDClassifier(random_state=17, n_jobs=-1)
v_for_sgd = TfidfVectorizer(encoding='unicode',min_df=1, ngram_range = (1, 2), max_df=0.9)
train = v_for_sgd.fit_transform(txts)
params = {
    'loss':['hinge', 'modified_huber', 'squared_hinge'],
    'class_weight':[None, 'balanced']
}
grid_cv = GridSearchCV(sgd, params, scoring = 'accuracy', cv = skf, n_jobs=-1)
grid_cv.fit(train, labs)
print grid_cv.best_params_
print i,j, grid_cv.best_score_

{'loss': 'hinge', 'class_weight': 'balanced'}
1 2 0.828625723417


In [204]:
sgd = SGDClassifier(random_state=17, n_jobs=-1, loss='hinge', class_weight='balanced')
sgd.fit(train, labs)
test = v_for_sgd.transform(data)
preds = sgd.predict(test)
writeanswer(preds, 'otvety_sgd.csv')

In [205]:
# настройка svc
svc = LinearSVC(random_state=17)
v_for_svc =TfidfVectorizer(encoding='unicode', analyzer='char', ngram_range=(2,5))
train = v_for_svc.fit_transform(txts)
params = {
    'loss': ['hinge', 'squared_hinge'],
    'C': [1,3,5],
    'class_weight': [None, 'balanced']
}
grid_cv = GridSearchCV(svc, params, scoring = 'accuracy', cv = skf, n_jobs=-1)
grid_cv.fit(train, labs)
print grid_cv.best_params_
print i,j, grid_cv.best_score_

{'loss': 'squared_hinge', 'C': 1, 'class_weight': None}
1 2 0.831476202816


In [206]:
svc = LinearSVC(random_state=17, C=1,loss='squared_hinge')
svc.fit(train, labs)
test = v_for_svc.transform(data)
preds = svc.predict(test)
writeanswer(preds, 'otvety_svc.csv')

In [240]:
svc = LinearSVC(random_state=17, C=1,loss='squared_hinge', class_weight='balanced')
train = v_for_svc.fit_transform(txts)
svc.fit(train, labs)
test = v_for_svc.transform(data)
preds = svc.predict(test)
writeanswer(preds, 'otvety_svc.csv')

**Результат на кегл: SVC - 0.76, SGD - 0.88 **

**Я сравнил ответы начального предсказания и предсказаний этих алгоримтов, их ошибки - считание негативных отзывов положительными в 95% случаев, это скорее всего связано с тем, что выборка не сбалансирована**

**Теперь попробуем случайный undersampling**

In [56]:
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(return_indices=False,random_state=17, ratio = {1:4000, -1:2000})

In [293]:
# логистическая регрессия
train = v_for_lr.fit_transform(txts)
test = v_for_lr.transform(data)
X_resampled, y_resampled = rus.fit_sample(train, labs)
lr = LogisticRegression(n_jobs=-1, C=1)

In [295]:
train.shape

(11577, 1026052)

In [309]:
# sgd 
train = v_for_sgd.fit_transform(txts)
test = v_for_sgd.transform(data)
X_resampled, y_resampled = rus.fit_sample(train, labs)
sgd = SGDClassifier(random_state=17, max_iter=1000,class_weight='balanced')

In [314]:
# svc
train = v_for_svc.fit_transform(txts)
test = v_for_svc.transform(data)
X_resampled, y_resampled = rus.fit_sample(train, labs)
svc = LinearSVC(class_weight='balanced')

**Много чего перепробовал, но все оказалось тщетно, поэтому решил проверить лучший результат (0.9) вручную. У него оказалось 9 ошибок ( 4 неправильных neg и 5 неправильных pos), то есть реальное (не кегл) accuracy 0.91, что не плохо, но и не очень хорошо. На этой выборке думаю большого качества уже не добиться, все остальные алгоритмы чаще исправляют правильные ответы чем неправильные для начального алгоритма. Попробуем добавить в нашу выборку отзывы с дробной оценкой (раньше их просто выкидывали), потому что непонятно куда отнести отзывы с оценкой 3,5 (скорее всего негативные??)**

In [481]:
def parsing_all(html):
    # создаем два массива для текстов и меток
    texts = []
    labels = []
    # сначала находим все отзывы на телефоны на 
    soup = bs4.BeautifulSoup(html)
    reviews = soup.find_all('div',{'class':'review-item__body'})
    # теперь для каждой рецензии
    for review in reviews:
        # находим оценку
        grade = review.find('span', {'class': 'review-item__rating-counter'})
        grade = grade.text
        grade = float(grade[0])
            
            # присваиваем ей класс
        if grade > 3:
            labels.append(1)
        else:
            labels.append(-1)
                # продолжаем в этом if-e. Дальше находим наш отзыв без недостатков и достоинтсв отдельно
                # он всегда пишется первым на этом сайте, поэтому используем просто find
        part = review.find('p', {'class': 'review-item__paragraph'})
                # теперь надо проверить есть ли ссылка "Читать полностью"
        link = part.find_all('a')
                # если ссылок нет загружаем текст отзыва
        if len(link) == 0:
            texts.append(part.text.strip())
                # если есть ссылка, достаем текст из нее
        else:
            txt = link[0].get('full-text')
            texts.append(txt.strip())
    return texts, labels

**Парсинг без мультипроцессинга занимает много времени! В следующих версиях программы постараюсь его добавить**

In [482]:
num = range(1,800)
txts = []
labs = []
for d in num:
    url = 'https://torg.mail.ru/review/goods/mobilephones/?page=%d' % (d)
    r = requests.get(url)
    t,l = parsing_all(r.text)
    txts.extend(t)
    labs.extend(l)

In [484]:
np.unique(labs, return_counts=True)

(array([-1,  1]), array([ 3586, 12394], dtype=int64))

In [603]:
with open('train_texts_16k.pkl', 'wb') as t_pkl:
    pickle.dump(txts, t_pkl, protocol=2)
with open('train_labs_16k.pkl', 'wb') as l_pkl:
    pickle.dump(labs, l_pkl, protocol=2)

In [584]:
tf = TfidfVectorizer()
a = tf.fit_transform(txts)
b = tf.transform(data)
lr = LogisticRegression(n_jobs=-1, C=1, class_weight='balanced')

In [489]:
lr = LogisticRegression(n_jobs=-1, C=1, class_weight='balanced')
lr.fit(a, labs)
preds = lr.predict(b)
writeanswer(preds,'otvety_16k.csv')

In [585]:
sgd = SGDClassifier(random_state=1, max_iter=1000,class_weight='balanced')
sgd.fit(a, labs)
preds = sgd.predict(b)
writeanswer(preds,'otvety_16k_sgd.csv')

**После долгих тестов были найдены лучшие для тестовой выборки алгоритмы. **

**Первый - обычный tfidf, random undersamler в пропорциях 33% на 66% и sgdclassifier. Kaggle - 0.94**

In [60]:
tf = TfidfVectorizer(ngram_range=(1,2))
a = tf.fit_transform(texts)
b = tf.transform(data)
sgd = SGDClassifier(random_state=10, max_iter=100,class_weight='balanced', loss='log')
sgd.fit(a, labs)

SGDClassifier(alpha=0.0001, average=False, class_weight='balanced',
       epsilon=0.1, eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', max_iter=100, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=10, shuffle=True,
       tol=None, verbose=0, warm_start=False)

In [61]:
with open('transform.pkl', 'wb') as X10_pkl:
    pickle.dump(tf, X10_pkl, protocol=2)
with open('classifier.pkl', 'wb') as X10_pkl:
    pickle.dump(sgd, X10_pkl, protocol=2)

**Такая же предобработка для логистической регрессии. Kaggle - 0.88**

In [587]:
lr = LogisticRegression(n_jobs=-1, C=1, class_weight='balanced')
lr.fit(X_resampled,y_resampled)
preds = lr.predict(b)
writeanswer(preds,'otvety_DRUlr.csv')

**Буквенные n-граммы и sgdclassifier. Kaggle - 0.98**

In [588]:
tf = TfidfVectorizer(ngram_range=(3,6), analyzer='char')
a = tf.fit_transform(txts)
b = tf.transform(data)
sgd = SGDClassifier(random_state=10, max_iter=100,class_weight='balanced')
sgd.fit(a, labs)
preds = sgd.predict(b)
writeanswer(preds,'otvety_chsgd.csv')

**Буквенные n-граммы и логистическая регрессия. Kaggle-0.98 **

In [589]:
tf = TfidfVectorizer(ngram_range=(3,5), analyzer='char')
a = tf.fit_transform(txts)
b = tf.transform(data)
lr = LogisticRegression(n_jobs=-1, C=1, class_weight='balanced')
lr.fit(a, labs)
preds = lr.predict(b)
writeanswer(preds,'otvety_chlr.csv')

In [604]:
tf = TfidfVectorizer(ngram_range=(3,5), analyzer='char')
a = tf.fit_transform(txts)
b = tf.transform(data)
lr = LogisticRegression(n_jobs=-1, C=1, class_weight='balanced')
print cross_val_score(lr, a, labs, cv=skf, scoring='accuracy')

[ 0.78078078  0.79740894  0.79703342]


In [605]:
tf = TfidfVectorizer(ngram_range=(3,6), analyzer='char')
a = tf.fit_transform(txts)
b = tf.transform(data)
sgd = SGDClassifier(random_state=10, max_iter=100,class_weight='balanced')
print cross_val_score(sgd, a, labs, cv=skf, scoring='accuracy')

[ 0.81343844  0.8218175   0.81656027]


In [609]:
tf = TfidfVectorizer(ngram_range=(1,4))
a = tf.fit_transform(txts)
b = tf.transform(data)
sgd = SGDClassifier(loss='log',random_state=10, max_iter=1000,class_weight='balanced')
print cross_val_score(sgd, a, labs, cv=skf, scoring='accuracy')

[ 0.82451201  0.81937664  0.83120541]


In [41]:
tf = TfidfVectorizer(ngram_range=(1,4))
a = tf.fit_transform(texts)

sgd = SGDClassifier(loss="log", n_jobs=-1, random_state=0, max_iter=1e4, tol=1e-4, class_weight="balanced")
print cross_val_score(sgd, a, labs, cv=skf, scoring='accuracy')

[ 0.82094595  0.81975216  0.83176868]


In [48]:
c = ['сломался, ужас а не телефон!']
d = tf.transform(c)
sgd.fit(a, labs)
sd = sgd.predict_proba(d)

In [49]:
print sd

[[ 0.83708943  0.16291057]]


**По моей разметке победили буквенные n-граммы и логистическая регрессия. Такой алгоритм показал 0.96 accuracy. Ниже выведены отзывы, для которых была допущена ошибка.**

In [591]:
print data[17]

Качество, глюки операционки.
Брендом нокиа пользуюсь более 10лет, было порядка 5 телефонов этой фирмы, в результате все чаще в качестве замены (пока новый телефон в сервисе) выступает старая раскладушка 6060 )) Качество увы не радует... У меня как раз телефон про-ва Индии, про которые уже много написано в отзывах что сборка там хромает на все лапы- плюс один.



**Классификатор могли сбить смайлы и не очень большое описание минусов**

In [594]:
print data[26]

Телефон хорош,звук громккий,фото достаточно качественные
Телефон очень хороший!советую



**Не понимаю какого типа этот отзыв, сарказм или нет? Поэтому просто записал его в ошибку**

In [597]:
print data[77]

Камера так себе, но снять что-нить забавное на улице вполне хватает.
К винде претензий нет, сыровата ещё. Но если не покупать на ней телефоны, то загнётся, а сама платформа интересная.



**Классификатор говорит, что это отрицательный отзыв, я думаю - положительный. Возможно смутило словосочетание "не покупать" и безграмотность автора**

In [601]:
print data[99]

ПО, камера, отсутствие выбора приложение, невозможность установки альтернативного браузера по умолчанию, тихий звук, регулировка громкости плеера-регулируешь общую громкость, виснет на "левой" музыке, и тд.
Приятный внешний вид и все. Убогость ОС убивает весь телефон. Подарил жене, повелся на красоту(синий), она довольна, но мне брать его в руки не хочется. пользователь сгс3.



**Классификатор думает, что это положительный отзыв, явно ошибочно**

**Еще раз повторюсь, что разметка сделана мной и она может ОТЛИЧАТЬСЯ от разметки, которую предоставил заказчик!! Однако считаю подобранный алгоритм более менее оптимальным, в конце концов мы же хотим, чтобы он мог предсказать тональность любого отзыва с хорошей точностью, а не только из тестовой сотни**

**Дальше черновик с небольшими тестами**

In [9]:
import pymorphy2
from nltk.corpus import stopwords
def clean_str(string):
    morph = pymorphy2.MorphAnalyzer()
    
    symbols = [
        ',', '.', '-', '*', '#', ')', '(', '/', '<', '>', ':', '+', '?', '!', '"', '"', '%', '=', '\\', '}'
    ]
    
    for symbol in symbols:
        string = str(string).replace(symbol, ' ')
        
    words = string.split()
    normalized_words = []
    
    for word in words:
        normalized_words.append(morph.parse(unicode(word.strip()))[0].normal_form)
        
    string = unicode(' '.join(normalized_words))
    
    return string

In [618]:
good_texts = []
for text in txts:
    good_texts.append(clean_str(text))

In [619]:
with open('train_good_texts_16k.pkl', 'wb') as t_pkl:
    pickle.dump(good_texts, t_pkl, protocol=2)

In [ ]:
from nltk.stem.snowball import RussianStemmer
stop_words= stopwords.words('russian')
stemmer = RussianStemmer(False)
stop_symbols = '.,:;"-\n\r'
nltk_texts = []
for text in txts:
    v=([stemmer.stem(x) for x in [y.strip(stop_symbols) for y in text.lower().split()] if x and (x not in stop_words)])
    d = ' '
    for word in v:
        d = d+word+" "
    nltk_texts.append(d)

In [ ]:
with open('train_nltk_texts_16k.pkl', 'wb') as t_pkl:
    pickle.dump(nltk_txts, t_pkl, protocol=2)

In [18]:
tf = TfidfVectorizer(ngram_range=(1, 3))
a = tf.fit_transform(good_t)
sgd = SGDClassifier(random_state=10, max_iter=1000,class_weight='balanced')
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=17)
print cross_val_score(sgd, a, labs, cv=skf, scoring='accuracy').mean()

0.827221466679


In [19]:
tf = TfidfVectorizer(ngram_range=(1, 2))
a = tf.fit_transform(good_t)
sgd = SGDClassifier(random_state=10, max_iter=1000,class_weight='balanced')
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=17)
print cross_val_score(sgd, a, labs, cv=skf, scoring='accuracy').mean()

0.826157550716


In [20]:
tf = TfidfVectorizer(ngram_range=(1, 2), stop_words=stopwords.words('russian'))
a = tf.fit_transform(good_t)
print cross_val_score(sgd, a, labs, cv=skf, scoring='accuracy')

[ 0.81737988  0.81693579  0.81806234]


In [22]:
tf = TfidfVectorizer()
a = tf.fit_transform(good_t)
print cross_val_score(sgd, a, labs, cv=skf, scoring='accuracy').mean()

0.804819300149


In [42]:
tf = TfidfVectorizer(stop_words=w, max_df=0.9)
a = tf.fit_transform(good_t)

In [622]:
print cross_val_score(sgd, a, labs, cv=skf, scoring='accuracy')

In [35]:
lr = LogisticRegression( random_state=17, class_weight='balanced')
print cross_val_score(lr, a, labs, cv=skf, scoring='accuracy')

In [2]:
import pickle
def openpickle(filename):
    with (open(filename, 'rb')) as openfile:
        objects = pickle.load(openfile)
    return objects

In [3]:
good_t = openpickle('train_good_texts_16k.pkl')
labs = openpickle('train_labs_16k.pkl')
texts = openpickle('train_texts_16k.pkl')

In [4]:
len(good_t), len(labs), len(texts)

(15980, 15980, 15980)

In [6]:
print texts[1]

Мне где то дней 10 назад,достался от сёстры,работает все кроме семерки,вот сейчас ищу виртуальную плаву на него)


In [7]:
tf = TfidfVectorizer()
lr = LogisticRegression(random_state=17,class_weight='balanced', n_jobs=-1)
sgd = SGDClassifier(random_state=10, class_weight='balanced', n_jobs=-1, max_iter=100)
skf = StratifiedKFold(n_splits=3, random_state=17, shuffle=True)
clfs = [lr,sgd]

In [10]:
w = stopwords.words('russian')
ngram = [(1,2),(1,3),(1,4), (1,1)]
stopwords = [None, w]
mind = [1,0.1,0.05]
maxd = [1.0, 0.95, 0.9]


In [11]:
res1 = []
for clf in clfs:
    print clf
    print
    for n in ngram:
        for s in stopwords:
            for mi in mind:
                for ma in maxd:
                    tf = TfidfVectorizer(ngram_range=n, stop_words=s, min_df=mi, max_df=ma)
                    a = tf.fit_transform(good_t)
                    b = cross_val_score(clf, a, labs, cv=skf, n_jobs=-1).mean()
                    res1.append(b)
                    print 'parametry: ', n,s,mi,ma,'result: ',b
                    print 

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=-1, penalty='l2', random_state=17,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

parametry:  (1, 2) None 1 1.0 result:  0.806133536841

parametry:  (1, 2) None 1 0.95 result:  0.806133536841

parametry:  (1, 2) None 1 0.9 result:  0.806133536841

parametry:  (1, 2) None 0.1 1.0 result:  0.65644601605

parametry:  (1, 2) None 0.1 0.95 result:  0.65644601605

parametry:  (1, 2) None 0.1 0.9 result:  0.65644601605

parametry:  (1, 2) None 0.05 1.0 result:  0.68704707336

parametry:  (1, 2) None 0.05 0.95 result:  0.68704707336

parametry:  (1, 2) None 0.05 0.9 result:  0.68704707336

parametry:  (1, 2) [u'\u0438', u'\u0432', u'\u0432\u043e', u'\u043d\u0435', u'\u0447\u0442\u043e', u'\u043e\u043d', u'\u043d\u0430', u'\u044f', u'\u0441', u'\u0441\u043e', u'\u043a\u0430\u043a', u'\u0430', u'\u0442\u043e',

parametry:  (1, 2) [u'\u0438', u'\u0432', u'\u0432\u043e', u'\u043d\u0435', u'\u0447\u0442\u043e', u'\u043e\u043d', u'\u043d\u0430', u'\u044f', u'\u0441', u'\u0441\u043e', u'\u043a\u0430\u043a', u'\u0430', u'\u0442\u043e', u'\u0432\u0441\u0435', u'\u043e\u043d\u0430', u'\u0442\u0430\u043a', u'\u0435\u0433\u043e', u'\u043d\u043e', u'\u0434\u0430', u'\u0442\u044b', u'\u043a', u'\u0443', u'\u0436\u0435', u'\u0432\u044b', u'\u0437\u0430', u'\u0431\u044b', u'\u043f\u043e', u'\u0442\u043e\u043b\u044c\u043a\u043e', u'\u0435\u0435', u'\u043c\u043d\u0435', u'\u0431\u044b\u043b\u043e', u'\u0432\u043e\u0442', u'\u043e\u0442', u'\u043c\u0435\u043d\u044f', u'\u0435\u0449\u0435', u'\u043d\u0435\u0442', u'\u043e', u'\u0438\u0437', u'\u0435\u043c\u0443', u'\u0442\u0435\u043f\u0435\u0440\u044c', u'\u043a\u043e\u0433\u0434\u0430', u'\u0434\u0430\u0436\u0435', u'\u043d\u0443', u'\u0432\u0434\u0440\u0443\u0433', u'\u043b\u0438', u'\u0435\u0441\u043b\u0438', u'\u0443\u0436\u0435', u'\u0438\u043b\u0438', u'

parametry:  (1, 2) [u'\u0438', u'\u0432', u'\u0432\u043e', u'\u043d\u0435', u'\u0447\u0442\u043e', u'\u043e\u043d', u'\u043d\u0430', u'\u044f', u'\u0441', u'\u0441\u043e', u'\u043a\u0430\u043a', u'\u0430', u'\u0442\u043e', u'\u0432\u0441\u0435', u'\u043e\u043d\u0430', u'\u0442\u0430\u043a', u'\u0435\u0433\u043e', u'\u043d\u043e', u'\u0434\u0430', u'\u0442\u044b', u'\u043a', u'\u0443', u'\u0436\u0435', u'\u0432\u044b', u'\u0437\u0430', u'\u0431\u044b', u'\u043f\u043e', u'\u0442\u043e\u043b\u044c\u043a\u043e', u'\u0435\u0435', u'\u043c\u043d\u0435', u'\u0431\u044b\u043b\u043e', u'\u0432\u043e\u0442', u'\u043e\u0442', u'\u043c\u0435\u043d\u044f', u'\u0435\u0449\u0435', u'\u043d\u0435\u0442', u'\u043e', u'\u0438\u0437', u'\u0435\u043c\u0443', u'\u0442\u0435\u043f\u0435\u0440\u044c', u'\u043a\u043e\u0433\u0434\u0430', u'\u0434\u0430\u0436\u0435', u'\u043d\u0443', u'\u0432\u0434\u0440\u0443\u0433', u'\u043b\u0438', u'\u0435\u0441\u043b\u0438', u'\u0443\u0436\u0435', u'\u0438\u043b\u0438', u'

parametry:  (1, 3) None 1 1.0 result:  0.819024361387

parametry:  (1, 3) None 1 0.95 result:  0.819024361387

parametry:  (1, 3) None 1 0.9 result:  0.819024361387

parametry:  (1, 3) None 0.1 1.0 result:  0.65644601605

parametry:  (1, 3) None 0.1 0.95 result:  0.65644601605

parametry:  (1, 3) None 0.1 0.9 result:  0.65644601605

parametry:  (1, 3) None 0.05 1.0 result:  0.68704707336

parametry:  (1, 3) None 0.05 0.95 result:  0.68704707336

parametry:  (1, 3) None 0.05 0.9 result:  0.68704707336

parametry:  (1, 3) [u'\u0438', u'\u0432', u'\u0432\u043e', u'\u043d\u0435', u'\u0447\u0442\u043e', u'\u043e\u043d', u'\u043d\u0430', u'\u044f', u'\u0441', u'\u0441\u043e', u'\u043a\u0430\u043a', u'\u0430', u'\u0442\u043e', u'\u0432\u0441\u0435', u'\u043e\u043d\u0430', u'\u0442\u0430\u043a', u'\u0435\u0433\u043e', u'\u043d\u043e', u'\u0434\u0430', u'\u0442\u044b', u'\u043a', u'\u0443', u'\u0436\u0435', u'\u0432\u044b', u'\u0437\u0430', u'\u0431\u044b', u'\u043f\u043e', u'\u0442\u043e\u043b

parametry:  (1, 3) [u'\u0438', u'\u0432', u'\u0432\u043e', u'\u043d\u0435', u'\u0447\u0442\u043e', u'\u043e\u043d', u'\u043d\u0430', u'\u044f', u'\u0441', u'\u0441\u043e', u'\u043a\u0430\u043a', u'\u0430', u'\u0442\u043e', u'\u0432\u0441\u0435', u'\u043e\u043d\u0430', u'\u0442\u0430\u043a', u'\u0435\u0433\u043e', u'\u043d\u043e', u'\u0434\u0430', u'\u0442\u044b', u'\u043a', u'\u0443', u'\u0436\u0435', u'\u0432\u044b', u'\u0437\u0430', u'\u0431\u044b', u'\u043f\u043e', u'\u0442\u043e\u043b\u044c\u043a\u043e', u'\u0435\u0435', u'\u043c\u043d\u0435', u'\u0431\u044b\u043b\u043e', u'\u0432\u043e\u0442', u'\u043e\u0442', u'\u043c\u0435\u043d\u044f', u'\u0435\u0449\u0435', u'\u043d\u0435\u0442', u'\u043e', u'\u0438\u0437', u'\u0435\u043c\u0443', u'\u0442\u0435\u043f\u0435\u0440\u044c', u'\u043a\u043e\u0433\u0434\u0430', u'\u0434\u0430\u0436\u0435', u'\u043d\u0443', u'\u0432\u0434\u0440\u0443\u0433', u'\u043b\u0438', u'\u0435\u0441\u043b\u0438', u'\u0443\u0436\u0435', u'\u0438\u043b\u0438', u'

...

parametry:  (1, 4) [u'\u0438', u'\u0432', u'\u0432\u043e', u'\u043d\u0435', u'\u0447\u0442\u043e', u'\u043e\u043d', u'\u043d\u0430', u'\u044f', u'\u0441', u'\u0441\u043e', u'\u043a\u0430\u043a', u'\u0430', u'\u0442\u043e', u'\u0432\u0441\u0435', u'\u043e\u043d\u0430', u'\u0442\u0430\u043a', u'\u0435\u0433\u043e', u'\u043d\u043e', u'\u0434\u0430', u'\u0442\u044b', u'\u043a', u'\u0443', u'\u0436\u0435', u'\u0432\u044b', u'\u0437\u0430', u'\u0431\u044b', u'\u043f\u043e', u'\u0442\u043e\u043b\u044c\u043a\u043e', u'\u0435\u0435', u'\u043c\u043d\u0435', u'\u0431\u044b\u043b\u043e', u'\u0432\u043e\u0442', u'\u043e\u0442', u'\u043c\u0435\u043d\u044f', u'\u0435\u0449\u0435', u'\u043d\u0435\u0442', u'\u043e', u'\u0438\u0437', u'\u0435\u043c\u0443', u'\u0442\u0435\u043f\u0435\u0440\u044c', u'\u043a\u043e\u0433\u0434\u0430', u'\u0434\u0430\u0436\u0435', u'\u043d\u0443', u'\u0432\u0434\u0440\u0443\u0433', u'\u043b\u0438', u'\u0435\u0441\u043b\u0438', u'\u0443\u0436\u0435', u'\u0438\u043b\u0438', u'

parametry:  (1, 1) None 1 0.9 result:  0.771464620836

parametry:  (1, 1) None 0.1 1.0 result:  0.655632232871

parametry:  (1, 1) None 0.1 0.95 result:  0.655632232871

parametry:  (1, 1) None 0.1 0.9 result:  0.655632232871

parametry:  (1, 1) None 0.05 1.0 result:  0.684667581588

parametry:  (1, 1) None 0.05 0.95 result:  0.684667581588

parametry:  (1, 1) None 0.05 0.9 result:  0.684667581588

parametry:  (1, 1) [u'\u0438', u'\u0432', u'\u0432\u043e', u'\u043d\u0435', u'\u0447\u0442\u043e', u'\u043e\u043d', u'\u043d\u0430', u'\u044f', u'\u0441', u'\u0441\u043e', u'\u043a\u0430\u043a', u'\u0430', u'\u0442\u043e', u'\u0432\u0441\u0435', u'\u043e\u043d\u0430', u'\u0442\u0430\u043a', u'\u0435\u0433\u043e', u'\u043d\u043e', u'\u0434\u0430', u'\u0442\u044b', u'\u043a', u'\u0443', u'\u0436\u0435', u'\u0432\u044b', u'\u0437\u0430', u'\u0431\u044b', u'\u043f\u043e', u'\u0442\u043e\u043b\u044c\u043a\u043e', u'\u0435\u0435', u'\u043c\u043d\u0435', u'\u0431\u044b\u043b\u043e', u'\u0432\u043e\

parametry:  (1, 1) [u'\u0438', u'\u0432', u'\u0432\u043e', u'\u043d\u0435', u'\u0447\u0442\u043e', u'\u043e\u043d', u'\u043d\u0430', u'\u044f', u'\u0441', u'\u0441\u043e', u'\u043a\u0430\u043a', u'\u0430', u'\u0442\u043e', u'\u0432\u0441\u0435', u'\u043e\u043d\u0430', u'\u0442\u0430\u043a', u'\u0435\u0433\u043e', u'\u043d\u043e', u'\u0434\u0430', u'\u0442\u044b', u'\u043a', u'\u0443', u'\u0436\u0435', u'\u0432\u044b', u'\u0437\u0430', u'\u0431\u044b', u'\u043f\u043e', u'\u0442\u043e\u043b\u044c\u043a\u043e', u'\u0435\u0435', u'\u043c\u043d\u0435', u'\u0431\u044b\u043b\u043e', u'\u0432\u043e\u0442', u'\u043e\u0442', u'\u043c\u0435\u043d\u044f', u'\u0435\u0449\u0435', u'\u043d\u0435\u0442', u'\u043e', u'\u0438\u0437', u'\u0435\u043c\u0443', u'\u0442\u0435\u043f\u0435\u0440\u044c', u'\u043a\u043e\u0433\u0434\u0430', u'\u0434\u0430\u0436\u0435', u'\u043d\u0443', u'\u0432\u0434\u0440\u0443\u0433', u'\u043b\u0438', u'\u0435\u0441\u043b\u0438', u'\u0443\u0436\u0435', u'\u0438\u043b\u0438', u'

parametry:  (1, 1) [u'\u0438', u'\u0432', u'\u0432\u043e', u'\u043d\u0435', u'\u0447\u0442\u043e', u'\u043e\u043d', u'\u043d\u0430', u'\u044f', u'\u0441', u'\u0441\u043e', u'\u043a\u0430\u043a', u'\u0430', u'\u0442\u043e', u'\u0432\u0441\u0435', u'\u043e\u043d\u0430', u'\u0442\u0430\u043a', u'\u0435\u0433\u043e', u'\u043d\u043e', u'\u0434\u0430', u'\u0442\u044b', u'\u043a', u'\u0443', u'\u0436\u0435', u'\u0432\u044b', u'\u0437\u0430', u'\u0431\u044b', u'\u043f\u043e', u'\u0442\u043e\u043b\u044c\u043a\u043e', u'\u0435\u0435', u'\u043c\u043d\u0435', u'\u0431\u044b\u043b\u043e', u'\u0432\u043e\u0442', u'\u043e\u0442', u'\u043c\u0435\u043d\u044f', u'\u0435\u0449\u0435', u'\u043d\u0435\u0442', u'\u043e', u'\u0438\u0437', u'\u0435\u043c\u0443', u'\u0442\u0435\u043f\u0435\u0440\u044c', u'\u043a\u043e\u0433\u0434\u0430', u'\u0434\u0430\u0436\u0435', u'\u043d\u0443', u'\u0432\u0434\u0440\u0443\u0433', u'\u043b\u0438', u'\u0435\u0441\u043b\u0438', u'\u0443\u0436\u0435', u'\u0438\u043b\u0438', u'

parametry:  (1, 1) [u'\u0438', u'\u0432', u'\u0432\u043e', u'\u043d\u0435', u'\u0447\u0442\u043e', u'\u043e\u043d', u'\u043d\u0430', u'\u044f', u'\u0441', u'\u0441\u043e', u'\u043a\u0430\u043a', u'\u0430', u'\u0442\u043e', u'\u0432\u0441\u0435', u'\u043e\u043d\u0430', u'\u0442\u0430\u043a', u'\u0435\u0433\u043e', u'\u043d\u043e', u'\u0434\u0430', u'\u0442\u044b', u'\u043a', u'\u0443', u'\u0436\u0435', u'\u0432\u044b', u'\u0437\u0430', u'\u0431\u044b', u'\u043f\u043e', u'\u0442\u043e\u043b\u044c\u043a\u043e', u'\u0435\u0435', u'\u043c\u043d\u0435', u'\u0431\u044b\u043b\u043e', u'\u0432\u043e\u0442', u'\u043e\u0442', u'\u043c\u0435\u043d\u044f', u'\u0435\u0449\u0435', u'\u043d\u0435\u0442', u'\u043e', u'\u0438\u0437', u'\u0435\u043c\u0443', u'\u0442\u0435\u043f\u0435\u0440\u044c', u'\u043a\u043e\u0433\u0434\u0430', u'\u0434\u0430\u0436\u0435', u'\u043d\u0443', u'\u0432\u0434\u0440\u0443\u0433', u'\u043b\u0438', u'\u0435\u0441\u043b\u0438', u'\u0443\u0436\u0435', u'\u0438\u043b\u0438', u'

In [12]:
res2 = []
for clf in clfs:
    print clf
    print
    for n in ngram:
        for s in stopwords:
            for mi in mind:
                for ma in maxd:
                    tf = TfidfVectorizer(ngram_range=n, stop_words=s, min_df=mi, max_df=ma)
                    a = tf.fit_transform(texts)
                    b = cross_val_score(clf, a, labs, cv=skf, n_jobs=-1).mean()
                    res2.append(b)
                    print 'parametry: ', n,s,mi,ma,'result: ',b
                    print 

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=-1, penalty='l2', random_state=17,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

parametry:  (1, 2) None 1 1.0 result:  0.811327874063

parametry:  (1, 2) None 1 0.95 result:  0.811327874063

parametry:  (1, 2) None 1 0.9 result:  0.811327874063

parametry:  (1, 2) None 0.1 1.0 result:  0.640802156832

parametry:  (1, 2) None 0.1 0.95 result:  0.640802156832

parametry:  (1, 2) None 0.1 0.9 result:  0.640802156832

parametry:  (1, 2) None 0.05 1.0 result:  0.677221579447

parametry:  (1, 2) None 0.05 0.95 result:  0.677221579447

parametry:  (1, 2) None 0.05 0.9 result:  0.677221579447

parametry:  (1, 2) [u'\u0438', u'\u0432', u'\u0432\u043e', u'\u043d\u0435', u'\u0447\u0442\u043e', u'\u043e\u043d', u'\u043d\u0430', u'\u044f', u'\u0441', u'\u0441\u043e', u'\u043a\u0430\u043a', u'\u0430', u'\u0442\u

parametry:  (1, 2) [u'\u0438', u'\u0432', u'\u0432\u043e', u'\u043d\u0435', u'\u0447\u0442\u043e', u'\u043e\u043d', u'\u043d\u0430', u'\u044f', u'\u0441', u'\u0441\u043e', u'\u043a\u0430\u043a', u'\u0430', u'\u0442\u043e', u'\u0432\u0441\u0435', u'\u043e\u043d\u0430', u'\u0442\u0430\u043a', u'\u0435\u0433\u043e', u'\u043d\u043e', u'\u0434\u0430', u'\u0442\u044b', u'\u043a', u'\u0443', u'\u0436\u0435', u'\u0432\u044b', u'\u0437\u0430', u'\u0431\u044b', u'\u043f\u043e', u'\u0442\u043e\u043b\u044c\u043a\u043e', u'\u0435\u0435', u'\u043c\u043d\u0435', u'\u0431\u044b\u043b\u043e', u'\u0432\u043e\u0442', u'\u043e\u0442', u'\u043c\u0435\u043d\u044f', u'\u0435\u0449\u0435', u'\u043d\u0435\u0442', u'\u043e', u'\u0438\u0437', u'\u0435\u043c\u0443', u'\u0442\u0435\u043f\u0435\u0440\u044c', u'\u043a\u043e\u0433\u0434\u0430', u'\u0434\u0430\u0436\u0435', u'\u043d\u0443', u'\u0432\u0434\u0440\u0443\u0433', u'\u043b\u0438', u'\u0435\u0441\u043b\u0438', u'\u0443\u0436\u0435', u'\u0438\u043b\u0438', u'

parametry:  (1, 2) [u'\u0438', u'\u0432', u'\u0432\u043e', u'\u043d\u0435', u'\u0447\u0442\u043e', u'\u043e\u043d', u'\u043d\u0430', u'\u044f', u'\u0441', u'\u0441\u043e', u'\u043a\u0430\u043a', u'\u0430', u'\u0442\u043e', u'\u0432\u0441\u0435', u'\u043e\u043d\u0430', u'\u0442\u0430\u043a', u'\u0435\u0433\u043e', u'\u043d\u043e', u'\u0434\u0430', u'\u0442\u044b', u'\u043a', u'\u0443', u'\u0436\u0435', u'\u0432\u044b', u'\u0437\u0430', u'\u0431\u044b', u'\u043f\u043e', u'\u0442\u043e\u043b\u044c\u043a\u043e', u'\u0435\u0435', u'\u043c\u043d\u0435', u'\u0431\u044b\u043b\u043e', u'\u0432\u043e\u0442', u'\u043e\u0442', u'\u043c\u0435\u043d\u044f', u'\u0435\u0449\u0435', u'\u043d\u0435\u0442', u'\u043e', u'\u0438\u0437', u'\u0435\u043c\u0443', u'\u0442\u0435\u043f\u0435\u0440\u044c', u'\u043a\u043e\u0433\u0434\u0430', u'\u0434\u0430\u0436\u0435', u'\u043d\u0443', u'\u0432\u0434\u0440\u0443\u0433', u'\u043b\u0438', u'\u0435\u0441\u043b\u0438', u'\u0443\u0436\u0435', u'\u0438\u043b\u0438', u'

parametry:  (1, 2) [u'\u0438', u'\u0432', u'\u0432\u043e', u'\u043d\u0435', u'\u0447\u0442\u043e', u'\u043e\u043d', u'\u043d\u0430', u'\u044f', u'\u0441', u'\u0441\u043e', u'\u043a\u0430\u043a', u'\u0430', u'\u0442\u043e', u'\u0432\u0441\u0435', u'\u043e\u043d\u0430', u'\u0442\u0430\u043a', u'\u0435\u0433\u043e', u'\u043d\u043e', u'\u0434\u0430', u'\u0442\u044b', u'\u043a', u'\u0443', u'\u0436\u0435', u'\u0432\u044b', u'\u0437\u0430', u'\u0431\u044b', u'\u043f\u043e', u'\u0442\u043e\u043b\u044c\u043a\u043e', u'\u0435\u0435', u'\u043c\u043d\u0435', u'\u0431\u044b\u043b\u043e', u'\u0432\u043e\u0442', u'\u043e\u0442', u'\u043c\u0435\u043d\u044f', u'\u0435\u0449\u0435', u'\u043d\u0435\u0442', u'\u043e', u'\u0438\u0437', u'\u0435\u043c\u0443', u'\u0442\u0435\u043f\u0435\u0440\u044c', u'\u043a\u043e\u0433\u0434\u0430', u'\u0434\u0430\u0436\u0435', u'\u043d\u0443', u'\u0432\u0434\u0440\u0443\u0433', u'\u043b\u0438', u'\u0435\u0441\u043b\u0438', u'\u0443\u0436\u0435', u'\u0438\u043b\u0438', u'

parametry:  (1, 3) [u'\u0438', u'\u0432', u'\u0432\u043e', u'\u043d\u0435', u'\u0447\u0442\u043e', u'\u043e\u043d', u'\u043d\u0430', u'\u044f', u'\u0441', u'\u0441\u043e', u'\u043a\u0430\u043a', u'\u0430', u'\u0442\u043e', u'\u0432\u0441\u0435', u'\u043e\u043d\u0430', u'\u0442\u0430\u043a', u'\u0435\u0433\u043e', u'\u043d\u043e', u'\u0434\u0430', u'\u0442\u044b', u'\u043a', u'\u0443', u'\u0436\u0435', u'\u0432\u044b', u'\u0437\u0430', u'\u0431\u044b', u'\u043f\u043e', u'\u0442\u043e\u043b\u044c\u043a\u043e', u'\u0435\u0435', u'\u043c\u043d\u0435', u'\u0431\u044b\u043b\u043e', u'\u0432\u043e\u0442', u'\u043e\u0442', u'\u043c\u0435\u043d\u044f', u'\u0435\u0449\u0435', u'\u043d\u0435\u0442', u'\u043e', u'\u0438\u0437', u'\u0435\u043c\u0443', u'\u0442\u0435\u043f\u0435\u0440\u044c', u'\u043a\u043e\u0433\u0434\u0430', u'\u0434\u0430\u0436\u0435', u'\u043d\u0443', u'\u0432\u0434\u0440\u0443\u0433', u'\u043b\u0438', u'\u0435\u0441\u043b\u0438', u'\u0443\u0436\u0435', u'\u0438\u043b\u0438', u'

...

parametry:  (1, 4) [u'\u0438', u'\u0432', u'\u0432\u043e', u'\u043d\u0435', u'\u0447\u0442\u043e', u'\u043e\u043d', u'\u043d\u0430', u'\u044f', u'\u0441', u'\u0441\u043e', u'\u043a\u0430\u043a', u'\u0430', u'\u0442\u043e', u'\u0432\u0441\u0435', u'\u043e\u043d\u0430', u'\u0442\u0430\u043a', u'\u0435\u0433\u043e', u'\u043d\u043e', u'\u0434\u0430', u'\u0442\u044b', u'\u043a', u'\u0443', u'\u0436\u0435', u'\u0432\u044b', u'\u0437\u0430', u'\u0431\u044b', u'\u043f\u043e', u'\u0442\u043e\u043b\u044c\u043a\u043e', u'\u0435\u0435', u'\u043c\u043d\u0435', u'\u0431\u044b\u043b\u043e', u'\u0432\u043e\u0442', u'\u043e\u0442', u'\u043c\u0435\u043d\u044f', u'\u0435\u0449\u0435', u'\u043d\u0435\u0442', u'\u043e', u'\u0438\u0437', u'\u0435\u043c\u0443', u'\u0442\u0435\u043f\u0435\u0440\u044c', u'\u043a\u043e\u0433\u0434\u0430', u'\u0434\u0430\u0436\u0435', u'\u043d\u0443', u'\u0432\u0434\u0440\u0443\u0433', u'\u043b\u0438', u'\u0435\u0441\u043b\u0438', u'\u0443\u0436\u0435', u'\u0438\u043b\u0438', u'

parametry:  (1, 4) [u'\u0438', u'\u0432', u'\u0432\u043e', u'\u043d\u0435', u'\u0447\u0442\u043e', u'\u043e\u043d', u'\u043d\u0430', u'\u044f', u'\u0441', u'\u0441\u043e', u'\u043a\u0430\u043a', u'\u0430', u'\u0442\u043e', u'\u0432\u0441\u0435', u'\u043e\u043d\u0430', u'\u0442\u0430\u043a', u'\u0435\u0433\u043e', u'\u043d\u043e', u'\u0434\u0430', u'\u0442\u044b', u'\u043a', u'\u0443', u'\u0436\u0435', u'\u0432\u044b', u'\u0437\u0430', u'\u0431\u044b', u'\u043f\u043e', u'\u0442\u043e\u043b\u044c\u043a\u043e', u'\u0435\u0435', u'\u043c\u043d\u0435', u'\u0431\u044b\u043b\u043e', u'\u0432\u043e\u0442', u'\u043e\u0442', u'\u043c\u0435\u043d\u044f', u'\u0435\u0449\u0435', u'\u043d\u0435\u0442', u'\u043e', u'\u0438\u0437', u'\u0435\u043c\u0443', u'\u0442\u0435\u043f\u0435\u0440\u044c', u'\u043a\u043e\u0433\u0434\u0430', u'\u0434\u0430\u0436\u0435', u'\u043d\u0443', u'\u0432\u0434\u0440\u0443\u0433', u'\u043b\u0438', u'\u0435\u0441\u043b\u0438', u'\u0443\u0436\u0435', u'\u0438\u043b\u0438', u'

parametry:  (1, 1) [u'\u0438', u'\u0432', u'\u0432\u043e', u'\u043d\u0435', u'\u0447\u0442\u043e', u'\u043e\u043d', u'\u043d\u0430', u'\u044f', u'\u0441', u'\u0441\u043e', u'\u043a\u0430\u043a', u'\u0430', u'\u0442\u043e', u'\u0432\u0441\u0435', u'\u043e\u043d\u0430', u'\u0442\u0430\u043a', u'\u0435\u0433\u043e', u'\u043d\u043e', u'\u0434\u0430', u'\u0442\u044b', u'\u043a', u'\u0443', u'\u0436\u0435', u'\u0432\u044b', u'\u0437\u0430', u'\u0431\u044b', u'\u043f\u043e', u'\u0442\u043e\u043b\u044c\u043a\u043e', u'\u0435\u0435', u'\u043c\u043d\u0435', u'\u0431\u044b\u043b\u043e', u'\u0432\u043e\u0442', u'\u043e\u0442', u'\u043c\u0435\u043d\u044f', u'\u0435\u0449\u0435', u'\u043d\u0435\u0442', u'\u043e', u'\u0438\u0437', u'\u0435\u043c\u0443', u'\u0442\u0435\u043f\u0435\u0440\u044c', u'\u043a\u043e\u0433\u0434\u0430', u'\u0434\u0430\u0436\u0435', u'\u043d\u0443', u'\u0432\u0434\u0440\u0443\u0433', u'\u043b\u0438', u'\u0435\u0441\u043b\u0438', u'\u0443\u0436\u0435', u'\u0438\u043b\u0438', u'

parametry:  (1, 1) [u'\u0438', u'\u0432', u'\u0432\u043e', u'\u043d\u0435', u'\u0447\u0442\u043e', u'\u043e\u043d', u'\u043d\u0430', u'\u044f', u'\u0441', u'\u0441\u043e', u'\u043a\u0430\u043a', u'\u0430', u'\u0442\u043e', u'\u0432\u0441\u0435', u'\u043e\u043d\u0430', u'\u0442\u0430\u043a', u'\u0435\u0433\u043e', u'\u043d\u043e', u'\u0434\u0430', u'\u0442\u044b', u'\u043a', u'\u0443', u'\u0436\u0435', u'\u0432\u044b', u'\u0437\u0430', u'\u0431\u044b', u'\u043f\u043e', u'\u0442\u043e\u043b\u044c\u043a\u043e', u'\u0435\u0435', u'\u043c\u043d\u0435', u'\u0431\u044b\u043b\u043e', u'\u0432\u043e\u0442', u'\u043e\u0442', u'\u043c\u0435\u043d\u044f', u'\u0435\u0449\u0435', u'\u043d\u0435\u0442', u'\u043e', u'\u0438\u0437', u'\u0435\u043c\u0443', u'\u0442\u0435\u043f\u0435\u0440\u044c', u'\u043a\u043e\u0433\u0434\u0430', u'\u0434\u0430\u0436\u0435', u'\u043d\u0443', u'\u0432\u0434\u0440\u0443\u0433', u'\u043b\u0438', u'\u0435\u0441\u043b\u0438', u'\u0443\u0436\u0435', u'\u0438\u043b\u0438', u'

parametry:  (1, 1) [u'\u0438', u'\u0432', u'\u0432\u043e', u'\u043d\u0435', u'\u0447\u0442\u043e', u'\u043e\u043d', u'\u043d\u0430', u'\u044f', u'\u0441', u'\u0441\u043e', u'\u043a\u0430\u043a', u'\u0430', u'\u0442\u043e', u'\u0432\u0441\u0435', u'\u043e\u043d\u0430', u'\u0442\u0430\u043a', u'\u0435\u0433\u043e', u'\u043d\u043e', u'\u0434\u0430', u'\u0442\u044b', u'\u043a', u'\u0443', u'\u0436\u0435', u'\u0432\u044b', u'\u0437\u0430', u'\u0431\u044b', u'\u043f\u043e', u'\u0442\u043e\u043b\u044c\u043a\u043e', u'\u0435\u0435', u'\u043c\u043d\u0435', u'\u0431\u044b\u043b\u043e', u'\u0432\u043e\u0442', u'\u043e\u0442', u'\u043c\u0435\u043d\u044f', u'\u0435\u0449\u0435', u'\u043d\u0435\u0442', u'\u043e', u'\u0438\u0437', u'\u0435\u043c\u0443', u'\u0442\u0435\u043f\u0435\u0440\u044c', u'\u043a\u043e\u0433\u0434\u0430', u'\u0434\u0430\u0436\u0435', u'\u043d\u0443', u'\u0432\u0434\u0440\u0443\u0433', u'\u043b\u0438', u'\u0435\u0441\u043b\u0438', u'\u0443\u0436\u0435', u'\u0438\u043b\u0438', u'

In [13]:
nchgr = [(2,5),(3,5),(3,6), (2,6),(4,6)]
for clf in clfs:
    print clf 
    print 
    for n in nchgr:
        tf = TfidfVectorizer(ngram_range=n, analyzer='char')
        a = tf.fit_transform(good_t)
        print n, cross_val_score(clf,a,labs,cv=skf,n_jobs=-1).mean()

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=-1, penalty='l2', random_state=17,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

(2, 5) 0.780789778701
(3, 5) 0.781102615007
(3, 6) 0.788299142176
(2, 6) 0.785045278098
(4, 6) 0.79136548302
SGDClassifier(alpha=0.0001, average=False, class_weight='balanced',
       epsilon=0.1, eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=100, n_iter=None,
       n_jobs=-1, penalty='l2', power_t=0.5, random_state=10, shuffle=True,
       tol=None, verbose=0, warm_start=False)

(2, 5) 0.790050635502
(3, 5) 0.792741553984
(3, 6) 0.80419351007
(2, 6) 0.802942000393
(4, 6) 0.807510054177


In [14]:
for clf in clfs:
    print clf 
    print 
    for n in nchgr:
        tf = TfidfVectorizer(ngram_range=n, analyzer='char')
        a = tf.fit_transform(texts)
        print n, cross_val_score(clf,a,labs,cv=skf,n_jobs=-1).mean()

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=-1, penalty='l2', random_state=17,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

(2, 5) 0.788799948089
(3, 5) 0.791741046341
(3, 6) 0.799375652408
(2, 6) 0.796371827141
(4, 6) 0.801315796645
SGDClassifier(alpha=0.0001, average=False, class_weight='balanced',
       epsilon=0.1, eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=100, n_iter=None,
       n_jobs=-1, penalty='l2', power_t=0.5, random_state=10, shuffle=True,
       tol=None, verbose=0, warm_start=False)

(2, 5) 0.803255071632
(3, 5) 0.805257144114
(3, 6) 0.81727206929
(2, 6) 0.814330806586
(4, 6) 0.81908699443
